In [ ]:
import tensorflow as tf
import numpy as np

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [2]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout
from tensorflow.keras.layers import multiply, concatenate, Activation, Masking, Reshape
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('gt_ver2.csv')

In [4]:
path = "./RR/" 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
def get_np_arrays(file_name):
    arr = pd.read_csv(f'{file_name}', header=None, names=['R'])['R'].values.reshape(-1, 1)
    
    seq_len = arr.shape[0]
    
    result = np.zeros([599, 1])
    result[:arr.shape[0],:arr.shape[1]] = arr
    
    #scaler.fit(result)
    #result = scaler.transform(result)
#     print(file_name)
#     arr = pd.read_csv(path + file_name)['# ECG Channel 1'].values
    return result

In [5]:
get_np_arrays('./RR/095-1_181|0.csv')

array([[168.],
       [168.],
       [166.],
       [153.],
       [179.],
       [165.],
       [165.],
       [165.],
       [164.],
       [151.],
       [164.],
       [178.],
       [151.],
       [179.],
       [165.],
       [155.],
       [180.],
       [167.],
       [166.],
       [151.],
       [176.],
       [159.],
       [157.],
       [156.],
       [152.],
       [137.],
       [149.],
       [148.],
       [162.],
       [148.],
       [137.],
       [153.],
       [168.],
       [157.],
       [159.],
       [162.],
       [148.],
       [177.],
       [165.],
       [165.],
       [169.],
       [165.],
       [169.],
       [168.],
       [169.],
       [176.],
       [169.],
       [172.],
       [170.],
       [169.],
       [168.],
       [169.],
       [167.],
       [167.],
       [166.],
       [155.],
       [179.],
       [166.],
       [166.],
       [151.],
       [176.],
       [163.],
       [162.],
       [162.],
       [163.],
       [162.],
       [14

In [6]:
class My_Custom_Generator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(599, 1),
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = get_np_arrays(ID)

            # Store class
            y[i] = df[df['file'] == ID]['class'].astype('float32')
#         return X, tf.keras.utils.to_categorical(y, num_classes=self.n_classes)        
        return np.asarray(X), np.asarray(y)

In [7]:
test_df = df[df['data_type'] == 'test']
train_df = df[df['data_type'] == 'train']
val_df = df[df['data_type'] == 'val']

In [8]:
# X_train = np.asarray([get_np_arrays('1.csv'), get_np_arrays('2.csv'), \
#                      get_np_arrays('8.csv'), get_np_arrays('9.csv'), \
#                      get_np_arrays('10.csv')])
# X_val = np.asarray([get_np_arrays('5.csv'), get_np_arrays('6.csv')])

In [9]:
X_train_filenames = train_df['file'].values.tolist()
# y_train = np.asarray([1, 0, 1, 0, 1])

X_val_filenames = val_df['file'].values.tolist()
# y_val = np.asarray([1, 0])

In [10]:
# X_train_filenames = train_df['file'].values.tolist()
# y_train = train_df['class'].values.tolist()

# X_val_filenames = val_df['file'].values.tolist()
# y_val = val_df['class'].values.tolist()

In [11]:
batch_size = 128

my_training_batch_generator = My_Custom_Generator(X_train_filenames, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_val_filenames, batch_size)

In [12]:
def get_LSTM_model():
    model = Sequential()
    model.add(LSTM(256, input_shape=(599, 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    return model

In [13]:
def squeeze_excite_block(input):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    filters = input.shape[-1] # channel_axis = -1 for TF

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se

In [14]:
def generate_model():
    ip = Input(shape=(599, 1))

    x = Masking()(ip)
    x = LSTM(8)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(2, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model

In [15]:
model2 = generate_model()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 599, 1)]     0                                            
__________________________________________________________________________________________________
permute (Permute)               (None, 1, 599)       0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 1, 128)       613504      permute[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1, 128)       512         conv1d[0][0]                     
_______________________________________________________________________________________

In [16]:
model = get_LSTM_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 264,449
Trainable params: 264,449
Non-trainable params: 0
_________________________________________________________________


In [17]:
adam = Adam(lr=0.1)
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model2.compile(optimizer=adam, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=200, batch_size=128, callbacks=[chk], validation_data=(X_val,y_val))
H = model2.fit(
    x=my_training_batch_generator,
    validation_data=my_validation_batch_generator,
    epochs=25, 
    batch_size=batch_size,
    callbacks=[chk])

Epoch 1/25
138/138 [==============================] - 52s 379ms/step - loss: 0.7590 - accuracy: 0.4995 - val_loss: 0.6954 - val_accuracy: 0.5735
Epoch 2/25
138/138 [==============================] - 52s 374ms/step - loss: 0.6934 - accuracy: 0.4652 - val_loss: 0.6948 - val_accuracy: 0.5771
Epoch 3/25
138/138 [==============================] - 51s 372ms/step - loss: 0.6933 - accuracy: 0.4782 - val_loss: 0.6949 - val_accuracy: 0.4226
Epoch 4/25
138/138 [==============================] - 51s 373ms/step - loss: 0.6936 - accuracy: 0.5267 - val_loss: 0.6932 - val_accuracy: 0.4221
Epoch 5/25
138/138 [==============================] - 51s 373ms/step - loss: 0.6931 - accuracy: 0.4939 - val_loss: 0.6933 - val_accuracy: 0.4229
Epoch 6/25
138/138 [==============================] - 52s 375ms/step - loss: 0.6931 - accuracy: 0.4779 - val_loss: 0.6934 - val_accuracy: 0.3030
Epoch 7/25
138/138 [==============================] - 52s 374ms/step - loss: 0.6931 - accuracy: 0.4728 - val_loss: 0.6935 - val_ac